In [ ]:
import os
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import plotly.graph_objs as go
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash.dash_table import DataTable
from datetime import datetime

try:
    firebase_app = firebase_admin.get_app()
except ValueError:
    cred_path = "weight-tracker-ae746-firebase-adminsdk-8gxqs-ee8f29cb2f.json"
    if not os.path.isfile(cred_path):
        raise FileNotFoundError(f"Service account key file not found at path: {cred_path}")
    cred = credentials.Certificate(cred_path)
    firebase_app = firebase_admin.initialize_app(cred)

db = firestore.client()

def fetch_all_weight_entries():
    users_ref = db.collection('users')
    all_weights = []
    
    for user in users_ref.stream():
        user_id = user.id
        weights_ref = users_ref.document(user_id).collection('weights')
        for weight_entry in weights_ref.stream():
            weight_data = weight_entry.to_dict()
            weight_data['user_id'] = user_id
            weight_data['entry_id'] = weight_entry.id
            all_weights.append(weight_data)
    
    return pd.DataFrame(all_weights)

def process_weight_data(df):
    df['date'] = pd.to_datetime(df['date'], utc=True)
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce')
    df = df.dropna(subset=['date', 'weight'])
    df = df.sort_values(['user_id', 'date'])
    return df

def calculate_total_weight_loss(df):
    return df.groupby('user_id').agg({
        'weight': ['first', 'last'],
        'date': ['first', 'last']
    }).reset_index()

def get_top_weight_losers(df, n=5):
    weight_loss = calculate_total_weight_loss(df)
    weight_loss['total_change'] = weight_loss[('weight', 'last')] - weight_loss[('weight', 'first')]
    weight_loss_sorted = weight_loss.sort_values(by='total_change')
    return weight_loss_sorted.head(n)

def plot_weight_trends(df, users=None, start_date=None, end_date=None):
    if users is not None and len(users) > 0:
        df = df[df['user_id'].isin(users)]
    if start_date is not None:
        df = df[df['date'] >= pd.to_datetime(start_date).tz_localize('UTC')]
    if end_date is not None:
        df = df[df['date'] <= pd.to_datetime(end_date).tz_localize('UTC').replace(hour=23, minute=59, second=59)]
    
    if df.empty:
        fig = go.Figure()
        fig.update_layout(title='No data available for the selected criteria.')
        return fig
    
    fig = go.Figure()
    for user in df['user_id'].unique():
        user_data = df[df['user_id'] == user].sort_values('date')
        fig.add_trace(go.Scatter(
            x=user_data['date'],
            y=user_data['weight'],
            mode='lines+markers',
            name=user
        ))
    
    fig.update_layout(
        title='Weight Trends',
        xaxis_title='Date',
        yaxis_title='Weight (kg)',
        hovermode='closest',
        xaxis_range=[df['date'].min(), df['date'].max()]
    )
    return fig

# fetch and prepare data
weight_df = fetch_all_weight_entries()
weight_df = process_weight_data(weight_df)
top_losers = get_top_weight_losers(weight_df)


top_losers_display = top_losers.copy()
top_losers_display.columns = ['User ID', 'Initial Weight', 'Latest Weight', 'Start Date', 'End Date', 'Total Change']
top_losers_display['Start Date'] = top_losers_display['Start Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
top_losers_display['End Date'] = top_losers_display['End Date'].dt.strftime('%Y-%m-%d %H:%M:%S')
top_losers_display['Total Change'] = top_losers_display['Total Change'].round(2)

# layout
dash_app = JupyterDash(__name__)

dash_app.layout = html.Div([
    html.H1("Weight Tracker Dashboard"),
    
    html.H2("Top Weight Losers"),
    DataTable(
        id='top-losers-table',
        columns=[{"name": i, "id": i} for i in top_losers_display.columns],
        data=top_losers_display.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left', 'padding': '5px', 'minWidth': '100px', 'width': '150px', 'maxWidth': '180px', 'whiteSpace': 'normal'},
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        page_size=10
    ),
    
    html.H2("Weight Trends"),
    html.Div([
        html.Label("Select Users:"),
        dcc.Dropdown(
            id='user-select',
            options=[{'label': uid, 'value': uid} for uid in weight_df['user_id'].unique()],
            multi=True,
            value=weight_df['user_id'].unique().tolist(),
            placeholder="Select users"
        )
    ], style={'width': '50%', 'display': 'inline-block', 'padding': '0 20px'}),
    
    html.Div([
        html.Label("Select Date Range:"),
        dcc.DatePickerRange(
            id='date-range',
            min_date_allowed=weight_df['date'].min().date(),
            max_date_allowed=weight_df['date'].max().date(),
            start_date=weight_df['date'].min().date(),
            end_date=weight_df['date'].max().date(),
            display_format='YYYY-MM-DD'
        )
    ], style={'marginTop': '20px', 'width': '50%', 'display': 'inline-block', 'padding': '0 20px'}),
    
    dcc.Graph(id='weight-trend-graph')
])

@dash_app.callback(
    Output('weight-trend-graph', 'figure'),
    [Input('user-select', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_graph(selected_users, start_date, end_date):
    filtered_df = weight_df.copy()
    
    if selected_users:
        filtered_df = filtered_df[filtered_df['user_id'].isin(selected_users)]
    
    if start_date:
        start_date = pd.to_datetime(start_date).tz_localize('UTC')
        filtered_df = filtered_df[filtered_df['date'] >= start_date]
    if end_date:
        end_date = pd.to_datetime(end_date).tz_localize('UTC').replace(hour=23, minute=59, second=59)
        filtered_df = filtered_df[filtered_df['date'] <= end_date]
    
    return plot_weight_trends(filtered_df)

if not weight_df.empty and not top_losers.empty:
    dash_app.run_server(mode='inline', debug=True)
else:
    print("not enough data to display.")